In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [33]:
# Paso 1: Preprocesamiento de datos
data_taxi = pd.read_csv(
    "C://Users//Andres//Desktop//AnyoneAI//Proyecto_Final//dataframe.csv"
)  # Reemplaza 'data_taxi.csv' con la ubicación correcta del archivo
data_taxi = data_taxi.dropna()  # Eliminar filas con valores faltantes o NaN

In [34]:
# Agregar la extracción de horas y dias de la semana

In [35]:
# Convierte la columna 'tpep_pickup_datetime' en formato datetime
data_taxi["tpep_pickup_datetime"] = pd.to_datetime(data_taxi["tpep_pickup_datetime"])
data_taxi["tpep_dropoff_datetime"] = pd.to_datetime(data_taxi["tpep_dropoff_datetime"])

data_taxi["duration"] = (
    data_taxi["tpep_dropoff_datetime"] - data_taxi["tpep_pickup_datetime"]
).dt.total_seconds() / 60

data_taxi["pickup_day"] = data_taxi["tpep_pickup_datetime"].dt.day
data_taxi["pickup_hour"] = data_taxi["tpep_pickup_datetime"].dt.hour

data_taxi["dropoff_day"] = data_taxi["tpep_dropoff_datetime"].dt.day
data_taxi["dropoff_hour"] = data_taxi["tpep_dropoff_datetime"].dt.hour

data_taxi = data_taxi.drop("Unnamed: 0", axis=1)

In [36]:
data_taxi

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,store_and_fwd_flag_encoded,pickup_day,pickup_hour,dropoff_day,dropoff_hour
0,1,2022-05-01 00:00:36,2022-05-01 00:19:18,1.0,6.6010,1.0,N,246,151,2,...,0.3,20.80,2.5,0.0,18.700000,0,1,0,1,0
1,1,2022-05-01 00:27:44,2022-05-01 00:41:33,1.0,3.7030,1.0,N,238,74,2,...,0.3,14.80,2.5,0.0,13.816667,0,1,0,1,0
2,1,2022-05-01 00:59:00,2022-05-01 01:14:22,1.0,6.7620,1.0,N,163,260,2,...,0.3,19.30,2.5,0.0,15.366667,0,1,0,1,1
3,1,2022-05-01 00:28:26,2022-05-01 00:37:49,1.0,2.5760,1.0,N,238,75,1,...,0.3,13.55,2.5,0.0,9.383333,0,1,0,1,0
4,2,2022-04-30 23:53:47,2022-05-01 00:05:17,1.0,3.0107,1.0,N,249,164,1,...,0.3,15.96,2.5,0.0,11.500000,0,30,23,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168605,2,2022-05-31 23:32:53,2022-05-31 23:38:24,1.0,2.1413,1.0,N,161,236,1,...,0.3,12.96,2.5,0.0,5.516667,0,31,23,31,23
3168606,2,2022-05-31 23:50:58,2022-05-31 23:57:27,1.0,3.0429,1.0,N,142,238,1,...,0.3,15.34,2.5,0.0,6.483333,0,31,23,31,23
3168607,2,2022-05-31 23:43:33,2022-05-31 23:57:09,1.0,5.5223,1.0,N,186,43,1,...,0.3,17.30,2.5,0.0,13.600000,0,31,23,31,23
3168608,2,2022-05-31 23:01:31,2022-05-31 23:04:42,2.0,1.5295,1.0,N,239,238,1,...,0.3,10.56,2.5,0.0,3.183333,0,31,23,31,23


In [37]:
(data_taxi.loc[:, data_taxi.columns[-9:]] < 0).sum()

total_amount                  0
congestion_surcharge          0
airport_fee                   0
duration                      0
store_and_fwd_flag_encoded    0
pickup_day                    0
pickup_hour                   0
dropoff_day                   0
dropoff_hour                  0
dtype: int64

In [38]:
data_taxi["RatecodeID"].value_counts()

RatecodeID
1.0    3033928
2.0     129395
5.0       4755
4.0        518
3.0          9
6.0          5
Name: count, dtype: int64

In [39]:
data_taxi = data_taxi.sample(10000, random_state=42)

In [40]:
(data_taxi.loc[:, data_taxi.columns[-18:]] < 0).sum()

PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge          0
airport_fee                   0
duration                      0
store_and_fwd_flag_encoded    0
pickup_day                    0
pickup_hour                   0
dropoff_day                   0
dropoff_hour                  0
dtype: int64

In [41]:
# Paso 2: Dividir los datos en características (X) y etiquetas (y)
X = data_taxi[
    [
        "VendorID",
        # "tpep_pickup_datetime",
        "PULocationID",
        "DOLocationID",
        "trip_distance",
        "passenger_count",
        "RatecodeID",
        "payment_type",
        "improvement_surcharge",
        "pickup_day",
        "pickup_hour",
    ]
]  # Características
y = data_taxi[["duration"]]  # Etiquetas

In [42]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Escalar los datos utilizando StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [43]:
model = Sequential(
    [
        Dense(32, activation="relu", input_shape=(X_train_scaled.shape[1],)),
        Dense(16, activation="relu"),
        Dense(1),
    ]
)

In [44]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [45]:
model.fit(
    X_train_scaled,
    y_train,
    epochs=10,
    batch_size=16,
    validation_data=(X_test_scaled, y_test),
)

Epoch 1/10
500/500 [==============================] - 2s 3ms/step - loss: 182.9974 - val_loss: 75.7616
Epoch 2/10
500/500 [==============================] - 1s 2ms/step - loss: 62.8083 - val_loss: 57.3747
Epoch 3/10
500/500 [==============================] - 1s 2ms/step - loss: 50.9329 - val_loss: 50.9612
Epoch 4/10
500/500 [==============================] - 1s 2ms/step - loss: 45.3893 - val_loss: 47.8543
Epoch 5/10
500/500 [==============================] - 1s 2ms/step - loss: 42.6300 - val_loss: 45.4907
Epoch 6/10
500/500 [==============================] - 1s 3ms/step - loss: 41.4170 - val_loss: 44.6254
Epoch 7/10
500/500 [==============================] - 1s 2ms/step - loss: 40.6593 - val_loss: 43.8212
Epoch 8/10
500/500 [==============================] - 1s 3ms/step - loss: 40.4517 - val_loss: 43.6134
Epoch 9/10
500/500 [==============================] - 1s 3ms/step - loss: 40.0477 - val_loss: 43.2225
Epoch 10/10
500/500 [==============================] - 1s 3ms/step - loss: 39.759

In [46]:
loss = model.evaluate(X_test_scaled, y_test)
print("Mean Squared Error:", loss)

63/63 [==============================] - 1s 2ms/step - loss: 43.0151
Mean Squared Error: 43.015079498291016


In [47]:
import xgboost as xgb

xgboost = xgb.XGBRegressor(random_state=42)

In [48]:
X_xgb = data_taxi[
    [
        "VendorID",
        # "tpep_pickup_datetime",
        "PULocationID",
        "DOLocationID",
        "trip_distance",
        "passenger_count",
        "RatecodeID",
        "payment_type",
        "improvement_surcharge",
        "pickup_day",
        "pickup_hour",
    ]
]  # Características
y_xgb = data_taxi[["duration"]]  # Etiquetas

In [49]:
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(
    X_xgb, y_xgb, test_size=0.2, random_state=42
)

In [50]:
xgboost.fit(X_test_xgb, y_test_xgb)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [51]:
y_pred = xgboost.predict(X_test_xgb)

In [52]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test_xgb, y_pred)
mae = mean_absolute_error(y_test_xgb, y_pred)
mae = r2_score(y_test_xgb, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R^2:", mae)

MSE: 1.4602320631563452
MAE: 0.9912419299924524
R^2: 0.9912419299924524


In [60]:
import pandas as pd
import xgboost as xgb
import ipywidgets as widgets
from IPython.display import display
from sklearn.model_selection import train_test_split

# Crear el modelo XGBoost
xgboost = xgb.XGBRegressor(random_state=42)

# Obtener las características y etiquetas del dataframe
X_xgb = data_taxi[
    [
        "VendorID",
        "PULocationID",
        "DOLocationID",
        "trip_distance",
        "passenger_count",
        "RatecodeID",
        "payment_type",
        "improvement_surcharge",
        "pickup_day",
        "pickup_hour",
    ]
]  # Características
y_xgb = data_taxi["duration"]  # Etiquetas

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(
    X_xgb, y_xgb, test_size=0.2, random_state=42
)

# Entrenar el modelo
xgboost.fit(X_train_xgb, y_train_xgb)

# Obtener los valores únicos de las características
vendor_id_values = X_xgb["VendorID"].unique()
pu_location_id_values = X_xgb["PULocationID"].unique()
do_location_id_values = X_xgb["DOLocationID"].unique()
trip_distance_values = X_xgb["trip_distance"].unique()
passenger_count_values = X_xgb["passenger_count"].unique()
ratecode_id_values = X_xgb["RatecodeID"].unique()
payment_type_values = X_xgb["payment_type"].unique()
improvement_surcharge_values = X_xgb["improvement_surcharge"].unique()
pickup_day_values = X_xgb["pickup_day"].unique()
pickup_hour_values = X_xgb["pickup_hour"].unique()


# Función para realizar la predicción con los datos ingresados
def predict_duration(
    vendor_id,
    pu_location_id,
    do_location_id,
    trip_distance,
    passenger_count,
    ratecode_id,
    payment_type,
    improvement_surcharge,
    pickup_day,
    pickup_hour,
):
    data = {
        "VendorID": [vendor_id],
        "PULocationID": [pu_location_id],
        "DOLocationID": [do_location_id],
        "trip_distance": [trip_distance],
        "passenger_count": [passenger_count],
        "RatecodeID": [ratecode_id],
        "payment_type": [payment_type],
        "improvement_surcharge": [improvement_surcharge],
        "pickup_day": [pickup_day],
        "pickup_hour": [pickup_hour],
    }
    input_data = pd.DataFrame(data)
    prediction = xgboost.predict(input_data)[0]
    return prediction


# Crea los widgets para cada una de las características de entrada
vendor_id_widget = widgets.IntSlider(
    min=1, max=2, step=1, value=1, description="VendorID:"
)
pu_location_id_widget = widgets.IntSlider(
    min=1, max=300, step=1, value=1, description="PULocationID:"
)
do_location_id_widget = widgets.IntSlider(
    min=1, max=300, step=1, value=1, description="DOLocationID:"
)
trip_distance_widget = widgets.FloatSlider(
    min=0.1, max=10.0, step=0.1, value=1.0, description="trip_distance:"
)
passenger_count_widget = widgets.FloatSlider(
    min=1.0, max=10.0, step=1.0, value=1.0, description="passenger_count:"
)
ratecode_id_widget = widgets.FloatSlider(
    min=0.0, max=10.0, step=1.0, value=0.0, description="RatecodeID:"
)
payment_type_widget = widgets.IntSlider(
    min=1, max=2, step=1, value=1, description="payment_type:"
)
improvement_surcharge_widget = widgets.FloatSlider(
    min=0.0, max=2.0, step=0.1, value=0.0, description="improvement_surcharge:"
)
pickup_day_widget = widgets.IntSlider(
    min=1, max=31, step=1, value=1, description="pickup_day:"
)
pickup_hour_widget = widgets.IntSlider(
    min=0, max=23, step=1, value=0, description="pickup_hour:"
)


# Función para interactuar con los widgets y realizar la predicción
def interact_fn(
    vendor_id,
    pu_location_id,
    do_location_id,
    trip_distance,
    passenger_count,
    ratecode_id,
    payment_type,
    improvement_surcharge,
    pickup_day,
    pickup_hour,
):
    prediction = predict_duration(
        vendor_id,
        pu_location_id,
        do_location_id,
        trip_distance,
        passenger_count,
        ratecode_id,
        payment_type,
        improvement_surcharge,
        pickup_day,
        pickup_hour,
    )
    return f"Predicted duration: {prediction:.2f} minutes"


# Crea el panel interactivo
panel = widgets.interact(
    interact_fn,
    vendor_id=vendor_id_widget,
    pu_location_id=pu_location_id_widget,
    do_location_id=do_location_id_widget,
    trip_distance=trip_distance_widget,
    passenger_count=passenger_count_widget,
    ratecode_id=ratecode_id_widget,
    payment_type=payment_type_widget,
    improvement_surcharge=improvement_surcharge_widget,
    pickup_day=pickup_day_widget,
    pickup_hour=pickup_hour_widget,
)

# Muestra el panel
display(panel)

interactive(children=(IntSlider(value=1, description='VendorID:', max=2, min=1), IntSlider(value=1, descriptio…

<function __main__.interact_fn(vendor_id, pu_location_id, do_location_id, trip_distance, passenger_count, ratecode_id, payment_type, improvement_surcharge, pickup_day, pickup_hour)>